# 이분적 유니온 파인드

노드 가의 관계가 2가지 상태만 있는 경우 빠르게 상태를 기준으로 나누는 자료구조

In [1]:
# 이분적 관계를 처리하는 Union-Find 클래스
# 각 노드는 루트와의 group(동일/반대 관계)를 유지하며 병합됨
from collections import defaultdict

class BipartiteUF:
  def __init__(self, size):
    self.parent = list(range(size))       # 각 노드의 대표(루트) 노드
    self.group = [0 for _ in range(size)] # 루트와의 관계 (0: 동일 그룹, 1: 반대 그룹)
    self.size = [1 for _ in range(size)]  # 루트가 관리하는 집합의 크기 (Small-to-Large 병합용)

  # 루트 노드를 찾고, 경로 압축과 group 갱신을 동시에 수행
  def find(self, x):
    px = self.parent[x]
    if px != x:
      self.parent[x] = self.find(px)  # 경로 압축: x의 부모를 루트로 설정
      self.group[x] ^= self.group[px] # group 갱신: x → 루트까지의 반대 관계 누적
    return self.parent[x]

  # x와 y를 같은 집합으로 병합 (is_enemy: 0 → 친구, 1 → 원수)
  def union(self, x, y, is_enemy):
    root_x = self.find(x)
    root_y = self.find(y)

    # 이미 같은 집합일 경우 → 현재 group와 원하는 관계가 충돌하는지 확인
    if root_x == root_y:
      return (self.group[x] ^ self.group[y]) == is_enemy

    # 작은 집합을 큰 집합에 병합 (Small-to-Large 최적화)
    if self.size[root_x] < self.size[root_y]:
      root_x, root_y = root_y, root_x
      x, y = y, x

    # 병합 수행: XOR을 통한 이분 관계 표현
    self.parent[root_y] = root_x
    self.group[root_y] = self.group[x] ^ self.group[y] ^ is_enemy
    self.size[root_x] += self.size[root_y]
    return True

  # 두 노드가 같은 집합에 속하는지 여부 (관계 무시)
  def is_connected(self, x, y):
    return self.find(x) == self.find(y)

  # 두 노드가 같은 그룹(동일 집합 + group 동일)인지 여부
  def is_same_group(self, x, y):
    if self.is_connected(x, y):
      return (self.group[x] ^ self.group[y]) == 0
  
  # 2개의 그룹에 대한 정보 (각 크기, 각 소속 원소)
  def print_group_info(self):
    info = defaultdict(lambda: {0: [], 1: []})

    for node in range(len(self.parent)):
      root = self.find(node)
      g = self.group[node]
      info[root][g].append(node)

    print("=== Group Info by Component ===")
    for root, groups in info.items():
      print(f'Component Root: {root}')
      print(f'  Group 0 (same as root): {groups[0]}')
      print(f'  Group 1 (opposite):     {groups[1]}')
      print()



In [2]:
uf = BipartiteUF(8)

# 친구 관계
uf.union(0, 1, 0)  # 0과 1은 친구 → 같은 그룹
uf.union(2, 3, 0)  # 2와 3은 친구 → 같은 그룹
uf.union(4, 5, 0)  # 4와 5는 친구 → 같은 그룹

# 원수 관계
uf.union(1, 2, 1)  # 1과 2는 원수 → 0,1과 2,3은 반대 그룹
uf.union(3, 4, 1)  # 3과 4는 원수 → 2,3과 4,5는 반대 그룹

# 그룹 정보 출력
uf.print_group_info()

=== Group Info by Component ===
Component Root: 0
  Group 0 (same as root): [0, 1, 4, 5]
  Group 1 (opposite):     [2, 3]

Component Root: 6
  Group 0 (same as root): [6]
  Group 1 (opposite):     []

Component Root: 7
  Group 0 (same as root): [7]
  Group 1 (opposite):     []

